# $ Wrangle$ $Linear$ $Regression$

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import wrangle as w

Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [2]:
df = w.get_zillow_data()

In [3]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [4]:
df = w.wrangle_single_family_residential()

In [5]:
df.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips'],
      dtype='object')

In [6]:
# squl query for acquisition
sql_query =  """
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
            FROM properties_2017
            Where propertylandusetypeid = 261;
        """
    # Acquisition
df = pd.read_sql(sql_query, env.get_connection('zillow'))

NameError: name 'env' is not defined

In [ ]:
df.sample(20)

<div class="alert alert-info"> There is 2152863 rows × 7 columns.
I can see NaN in calculatedfinishedsquarefeet,yearbuilt,taxamount

In [ ]:
def plot_boxplot(df,feature):
    df.boxplot(column=[feature])
    plt.grid(False)
    Q1 = df[feature].quantile(.25)
    Q2 = df[feature].quantile(.5)
    Q3 = df[feature].quantile(.75)
    Q4 = df[feature].quantile(1)
    IQR =  Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    print ('Q1:', Q1)
    print ('Q2:', Q2)
    print ('Q3:', Q3)
    print ('Q4:', Q4)
    print ('IQR', IQR)
    print('lower bound:',lower_bound)
    print('upper bound:' ,upper_bound)
    plt.show()

### Look into bedroom count

In [ ]:
df.bedroomcnt.max()

In [ ]:
df[df.bedroomcnt > df.bedroomcnt.quantile(.75)]

In [ ]:
plot_boxplot(df,'bedroomcnt')

In [ ]:
df.bedroomcnt.value_counts( sort=True, bins = 5)

In [ ]:
(6)/(len(df))* 100,(75 +6)/(2152852)* 100

<div class="alert alert-info"> 
6 rows are 15 bedrooms and above that is 0.0003% of total data
    
75 + 6 rows are 10 bedrooms and above that is 0.004% of total data
    
If I where to remove outliers I might set parameters for outliers to be 15+ (revmoves 6 rows) or 10+(removes 81 rows) 

In [ ]:
# bedrooms with more than 10 rooms
df[df.bedroomcnt>10]

### Look into bathroom count

In [ ]:
plot_boxplot(df,'bathroomcnt')

In [ ]:
df.bathroomcnt.value_counts().sort_values

In [ ]:
df.bathroomcnt.value_counts(normalize=False, bins =16)

In [ ]:
51/2152852 *100

<div class="alert alert-info"> ????? not sure on this one
21 rows are 16 bathrooms and above that 0.0007% of total data
    
51 rows are 14 bath rooms and above that 0.002% of total data

    
If I where to remove outliers I might set parameters for outliers to be 19.5+ (removes 21 rows) or 14 (removes 76  rows) 

In [ ]:
# bathrooms with more than 10 rooms
df.bathroomcnt[df.bathroomcnt>16].value_counts(bins=1)

In [ ]:
def outliers (df,feature):
    Q1 = df[feature].quantile(.25)
    Q3 = df[feature].quantile(.75)
    IQR =  Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    ls = df[(df[feature]<lower_bound) | (df[feature] > upper_bound)]
    return ls

In [ ]:
df[df.yearbuilt.isnull()]

<div class="alert alert-info"> 
Many values that have a year built NaN also have a bedroomcnt = 0, bathroomcnt = 0 and squarefeet NaN,
     9337 rows 

In [ ]:
df[(df.yearbuilt.isnull()) & (df.bathroomcnt.isnull())& (df.calculatedfinishedsquarefeet.isnull())]

<div class="alert alert-info"> 
There is three rows that have NaN in yearbuilt, bathrommcnt, bedroomcnt. and squarefeet

In [ ]:
outliers_bathroom = outliers(df, 'bathroomcnt') 

In [ ]:
outliers_bedroom = outliers(df, 'bedroomcnt') 

In [ ]:
outliers_squarefeet = outliers(df, 'calculatedfinishedsquarefeet') 

In [ ]:
outliers_bathroom = outliers(df, 'bathroomcnt') 

In [ ]:
plot_boxplot(df,'bathroomcnt')

In [ ]:
# to show all value counts in info
df.info(show_counts=  True)

info() did not mark the null values. All columns are dataType- float64

In [ ]:
df.isnull().any()

In [ ]:
df.isna().sum()

<div class="alert alert-info">
Null values
    
* 11 values in bedroomcnt 
* 11 values in bathroomcnt
* 8484 values in calculatedfinishedsquarefeet
* 493 values in taxvaluedollarcnt
* 9337 values in yearbuilt 
* 4442 values in taxamount

No null values in fips

In [ ]:
df.nunique()

In [ ]:
df.size, df.shape

In [ ]:
df.describe().T

<div class="alert alert-info">
    
Bedroom range 0 to 25
    
bathroom range 0 to 32
    
square feet range 1 to 952576.00
    
Year built has a range from 1801 to 2016
    
Tax amount has a range from 1.85 to 1337755.86
 

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
2140235-2152863

<div class="alert alert-info">
If all null values are dropped we lose 12628 rows

In [ ]:
def get_zillow_data():
        ''' Acquire Zillow data using properties_2017 table from Code up Data Base. Columns bedroomcnt, 
            bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips 
        '''
   
    # sql query for acquisition
        sql_query = """
        SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        FROM properties_2017
        Where propertylandusetypeid = 261;
        """
    # Acquisition
        df = pd.read_sql(sql_query, env.get_connection('zillow'))

In [ ]:
def wrangle_single_family_residential():
    '''Wrangle_single_family_residential checks to see if the csv or zillow data exits. If it does not
        it will call for the data and save a csv file. It will return a dataframe that had drop all nulls.
    '''
    # obtain csv file
    if os.path.isfile('zillow.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = get_zillow_data()
        
        # Cache data
        df.to_csv('zillow.csv')
    # replace blank spaces and special characters
    df = df.replace(r'^\s*$', np.nan, regex = True)
    
    df = df.dropna()
    return df

In [ ]:
df = w. wrangle_single_family_residential()

In [ ]:
df

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(25, 5))

for i, col in enumerate(df):
    
    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1
    
    # Create subplot.
    # plt.subplot(row X col, where?)
    plt.subplot(1,7,plot_number)
    
    # Title with column name.
    plt.title(col)
    
    # Display histogram for column.
    df.boxplot(column=[col])
    
    # Hide gridlines.
    plt.grid(False)

In [ ]:
df.describe().T

In [ ]:
df.shape